In [24]:
# Import necessary libraries
import os
from textblob import TextBlob
from docx import Document  # Ensure this import statement is included


In [25]:
# Function to get sentiment
def get_sentiment(text):
    return TextBlob(text).sentiment.polarity


In [26]:
# Function to extract text from a .docx file
def extract_text_from_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)


In [27]:
# Function to analyze content related to user-specified keywords
def analyze_content(text, keywords):
    # Convert to lowercase for easier matching
    content_lower = text.lower()

    # Check if any keyword is in the content
    if not any(keyword.lower() in content_lower for keyword in keywords):
        print(f"No content related to {', '.join(keywords)} found in the document.")
        return

    # Split content into sentences
    sentences = text.split('.')

    # Analyze each sentence
    for sentence in sentences:
        # Check if sentence contains any of the keywords
        if any(keyword.lower() in sentence.lower() for keyword in keywords):
            # Clean the sentence
            clean_sentence = sentence.strip()
            
            # Get sentiment
            sentiment = get_sentiment(clean_sentence)

            # Interpret sentiment
            if sentiment > 0.05:
                sentiment_label = "Positive"
            elif sentiment < -0.05:
                sentiment_label = "Negative"
            else:
                sentiment_label = "Neutral"

            # Check for specific keywords indicating price movement or trading activity
            price_up = any(word in clean_sentence.lower() for word in ['rise', 'rising', 'up', 'bull', 'bullish', 'increase'])
            price_down = any(word in clean_sentence.lower() for word in ['fall', 'falling', 'down', 'bear', 'bearish', 'decrease'])
            buying = 'buy' in clean_sentence.lower() or 'buying' in clean_sentence.lower()
            selling = 'sell' in clean_sentence.lower() or 'selling' in clean_sentence.lower()

            # Print results
            print(f"Sentence: {clean_sentence}")
            print(f"Sentiment: {sentiment_label} ({sentiment:.2f})")
            if price_up:
                print("Indicates price might be going up")
            if price_down:
                print("Indicates price might be going down")
            if buying:
                print("Mentions buying activity")
            if selling:
                print("Mentions selling activity")
            print()


In [28]:
# Prompt user for input
keywords_input = input("Enter the keywords to analyze (comma-separated): ")
keywords = [keyword.strip() for keyword in keywords_input.split(',')]

# Extract text from the uploaded document
file_path = 'btcSediment.docx'
document_text = extract_text_from_docx(file_path)

# Analyze the extracted text
analyze_content(document_text, keywords)

Sentence: The price of bitcoin (BTC-USD) is up more than 13% over the past month and surged above $68,000 in the last 24 hours, putting the world’s-largest cryptocurrency within striking distance of an all-time high set earlier this year
Sentiment: Positive (0.13)
Indicates price might be going up

Sentence: This week crypto investors are anticipating several reasons to get more hyped, including Securities and Exchange Commission approvals for big Wall Street money managers to soon issue exchange-traded funds that hold ether (ETH-USD), the world’s second-largest cryptocurrency
Sentiment: Positive (0.17)

Sentence: "

Trump, who in the past has called bitcoin a "scam" referred to cryptocurrencies as "amazing" in an interview with Bloomberg published last week
Sentiment: Positive (0.12)

Sentence: Republican presidential candidate former President Donald Trump reacts after speaking at a campaign rally, Saturday, July 20, 2024, in Grand Rapids, Mich
Sentiment: Positive (0.25)

Sentence: (

In [29]:
# Prompt user for input
keywords_input = input("Enter the keywords to analyze (comma-separated): ")
keywords = [keyword.strip() for keyword in keywords_input.split(',')]

# Extract text from the uploaded document
file_path = 'btcSediment.docx'
document_text = extract_text_from_docx(file_path)


In [30]:
# Analyze the extracted text
analyze_content(document_text, keywords)


Sentence: The price of bitcoin (BTC-USD) is up more than 13% over the past month and surged above $68,000 in the last 24 hours, putting the world’s-largest cryptocurrency within striking distance of an all-time high set earlier this year
Sentiment: Positive (0.13)
Indicates price might be going up

Sentence: This week crypto investors are anticipating several reasons to get more hyped, including Securities and Exchange Commission approvals for big Wall Street money managers to soon issue exchange-traded funds that hold ether (ETH-USD), the world’s second-largest cryptocurrency
Sentiment: Positive (0.17)

Sentence: "

Trump, who in the past has called bitcoin a "scam" referred to cryptocurrencies as "amazing" in an interview with Bloomberg published last week
Sentiment: Positive (0.12)

Sentence: Republican presidential candidate former President Donald Trump reacts after speaking at a campaign rally, Saturday, July 20, 2024, in Grand Rapids, Mich
Sentiment: Positive (0.25)

Sentence: (

In [31]:
def generate_report(sentences, sentiments, keywords):
    report = {
        "Positive": [],
        "Negative": [],
        "Neutral": []
    }
    price_up_count = 0
    price_down_count = 0
    buying_count = 0
    selling_count = 0

    for sentence, sentiment in zip(sentences, sentiments):
        sentiment_label = "Neutral"
        if sentiment > 0.05:
            sentiment_label = "Positive"
            report["Positive"].append((sentence, sentiment))
        elif sentiment < -0.05:
            sentiment_label = "Negative"
            report["Negative"].append((sentence, sentiment))
        else:
            report["Neutral"].append((sentence, sentiment))

        if any(word in sentence.lower() for word in ['rise', 'rising', 'up', 'bull', 'bullish', 'increase','great', 'possible', 'ETF']):
            price_up_count += 1
        if any(word in sentence.lower() for word in ['fall', 'falling', 'down', 'bear', 'bearish', 'decrease', 'bad', 'decline', 'declining', 'terrible']):
            price_down_count += 1
        if 'buy' in sentence.lower() or 'buying' in sentence.lower():
            buying_count += 1
        if 'sell' in sentence.lower() or 'selling' in sentence.lower():
            selling_count += 1

    summary = {
        "Positive Sentences": len(report["Positive"]),
        "Negative Sentences": len(report["Negative"]),
        "Neutral Sentences": len(report["Neutral"]),
        "Price Up Indications": price_up_count,
        "Price Down Indications": price_down_count,
        "Buying Activity Mentions": buying_count,
        "Selling Activity Mentions": selling_count
    }

    return report, summary


In [32]:
# Example usage
sentences = [
    "Crypto is surging again on a new wave of political respect...",
    "The price of bitcoin (BTC-USD) is up more than 13% over the past month...",
    # Add other sentences here
]
sentiments = [0.05, 0.13, 0.00, 0.20, 0.25, 0.21, 0.00, 0.50]

report, summary = generate_report(sentences, sentiments, keywords)

print("Summary:")
for key, value in summary.items():
    print(f"{key}: {value}")

print("\nDetailed Report:")
for sentiment, details in report.items():
    print(f"\n{sentiment} Sentiment:")
    for sentence, score in details:
        print(f"Sentence: {sentence}")
        print(f"Sentiment Score: {score:.2f}")
        print()


Summary:
Positive Sentences: 1
Negative Sentences: 0
Neutral Sentences: 1
Price Up Indications: 1
Price Down Indications: 0
Buying Activity Mentions: 0
Selling Activity Mentions: 0

Detailed Report:

Positive Sentiment:
Sentence: The price of bitcoin (BTC-USD) is up more than 13% over the past month...
Sentiment Score: 0.13


Negative Sentiment:

Neutral Sentiment:
Sentence: Crypto is surging again on a new wave of political respect...
Sentiment Score: 0.05

